# 4.4 LSTM on Return Close Target

In this notebook we will attempt to learn on the decision labels 'D2', which have a 50/50 buy/sell split without hold positions indicated. We hope this will be an easier decision function to learn, rather than the 3 class classification of buy/hold/sell. 

In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np

In [2]:
y = np.load('./data/prepared/august25screenfixed/numpy_matrices/y_br.npy')

In [3]:
y.shape

(30, 4000)

In [4]:
y_train = y[:, :2000]
y_test = y[:, 2000:]

y_train.shape, y_test.shape

((30, 2000), (30, 2000))

In [5]:
y_train = y_train.reshape([60000, 1])
y_test = y_test.reshape([60000, 1])
y_train.shape, y_test.shape

((60000, 1), (60000, 1))

In [6]:
X_train = np.load('./data/prepared/august25screenfixed/numpy_matrices/X_train.npy')
X_test = np.load('./data/prepared/august25screenfixed/numpy_matrices/X_test.npy')

In [7]:
X_train.shape, X_test.shape

((60000, 1, 116, 60), (60000, 1, 116, 60))

In [8]:
np.unique(y_train, return_counts=True)

(array([-0.10514541, -0.06351351, -0.06047963, ...,  0.06489185,
         0.06741573,  0.12470588]),
 array([1, 1, 1, ..., 1, 1, 1]))

In [9]:
np.unique(y_test, return_counts=True)

(array([-0.07625272, -0.061875  , -0.06145251, ...,  0.09659091,
         0.09957627,  0.10841704]),
 array([1, 1, 1, ..., 1, 1, 1]))

In [11]:
# Undersample to balance classes for training set ?
b = np.where(y_train == 1)[0]
s = np.where(y_train == 2)[0]

bi = np.random.choice(b, size=21000, replace=False)
si = np.random.choice(s, size=21000, replace=False)

ind = np.concatenate([bi,si])
ind.shape

ValueError: 'a' cannot be empty unless no samples are taken

In [10]:
np.unique(y_train[ind], return_counts=True)

NameError: name 'ind' is not defined

In [ ]:
X_train[ind].shape

In [ ]:
y_train = y_train[ind]
X_train = X_train[ind]

In [11]:
X_train.shape, y_train.shape

((60000, 1, 116, 60), (60000, 1))

In [10]:
X_train = X_train[:,:,:,:10]
X_train.shape

(60000, 1, 116, 10)

In [11]:
X_train.shape = (60000, 116, 10)
X_train.shape

(60000, 116, 10)

In [ ]:
# decrease test size for runtime and memory concerns
b = np.where(y_test == 1)[0]
s = np.where(y_test == 2)[0]

bi = np.random.choice(b, size=20000, replace=False)
si = np.random.choice(s, size=20000, replace=False)

indt = np.concatenate([bi,si])
indt.shape

y_test = y_test[indt]
X_test = X_test[indt]
y_test.shape, X_test.shape

In [12]:
X_test = X_test[:,:,:,:10]
X_test.shape = (60000, 116, 10)
X_test.shape

(60000, 116, 10)

In [ ]:
np.unique(y_test, return_counts=True)[1]/y_test.shape[0]

In [ ]:
np.unique(y_train, return_counts=True),np.unique(y_test, return_counts=True)

In [ ]:
y_train[np.where(y_train == 2)] = 0
y_test[np.where(y_test == 2)] = 0

np.unique(y_train, return_counts=True),np.unique(y_test, return_counts=True)

#### LSTM Testing

In [13]:
Xr_tp = np.transpose(X_train, axes=(0,2,1))
Xr_tp.shape

(60000, 10, 116)

In [14]:
Xe_tp = np.transpose(X_test, axes=(0,2,1))
Xe_tp.shape

(60000, 10, 116)

In [24]:
model = keras.Sequential([
    keras.layers.LSTM(1),
    keras.layers.Dense(1)
    
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.MeanSquaredError(),
              metrics=['mean_squared_error'])

model.fit(Xr_tp, yr_b, epochs=5,
         validation_data=(Xe_tp, ye_b))

Train on 60000 samples, validate on 60000 samples
Epoch 1/50
60000/60000 [==============================] - 17s 291us/sample - loss: 819.2574 - mean_squared_error: 819.2581 - val_loss: 770.8177 - val_mean_squared_error: 770.8179
Epoch 2/50
60000/60000 [==============================] - 14s 238us/sample - loss: 819.1736 - mean_squared_error: 819.1735 - val_loss: 770.9350 - val_mean_squared_error: 770.9357
Epoch 3/50
60000/60000 [==============================] - 14s 232us/sample - loss: 819.1198 - mean_squared_error: 819.1199 - val_loss: 770.9629 - val_mean_squared_error: 770.9623
Epoch 4/50
60000/60000 [==============================] - 16s 261us/sample - loss: 819.0930 - mean_squared_error: 819.0927 - val_loss: 770.8600 - val_mean_squared_error: 770.8605
Epoch 5/50
60000/60000 [==============================] - 15s 257us/sample - loss: 819.0596 - mean_squared_error: 819.0594 - val_loss: 770.8098 - val_mean_squared_error: 770.8100
Epoch 6/50
60000/60000 [==============================]

KeyboardInterrupt: 

In [23]:
yr_b = y_train*10**4
ye_b = y_test*10**4

In [15]:
yr_d = y_train.copy()
yr_d[np.where(yr_d > 0)] = 1
yr_d[np.where(yr_d < 0)] = 0
np.unique(yr_d, return_counts=True)

(array([0., 1.]), array([32831, 27169]))

In [16]:
ye_d = y_test.copy()
ye_d[np.where(ye_d > 0)] = 1
ye_d[np.where(ye_d < 0)] = 0
np.unique(ye_d, return_counts=True)

(array([0., 1.]), array([33401, 26599]))

In [17]:
model = keras.Sequential([
    keras.layers.LSTM(10),
    keras.layers.Dense(2, activation='softmax')
    
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(Xr_tp, yr_d, epochs=10,
         validation_data=(Xe_tp, ye_d))

Train on 60000 samples, validate on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 14s 240us/sample - loss: 0.6870 - accuracy: 0.5447 - val_loss: 0.6835 - val_accuracy: 0.5563
Epoch 2/10
60000/60000 [==============================] - 13s 209us/sample - loss: 0.6848 - accuracy: 0.5456 - val_loss: 0.6820 - val_accuracy: 0.5563
Epoch 3/10
60000/60000 [==============================] - 13s 210us/sample - loss: 0.6837 - accuracy: 0.5469 - val_loss: 0.6832 - val_accuracy: 0.5567
Epoch 4/10
60000/60000 [==============================] - 13s 215us/sample - loss: 0.6832 - accuracy: 0.5474 - val_loss: 0.6818 - val_accuracy: 0.5579
Epoch 5/10
60000/60000 [==============================] - 13s 217us/sample - loss: 0.6821 - accuracy: 0.5518 - val_loss: 0.6800 - val_accuracy: 0.5624
Epoch 6/10
60000/60000 [==============================] - 13s 212us/sample - loss: 0.6803 - accuracy: 0.5596 - val_loss: 0.6793 - val_accuracy: 0.5610
Epoch 7/10
60000/60000 [====================

In [18]:
np.unique(ye_d, return_counts=True)[1]/np.shape(ye_d)[0]

array([0.55668333, 0.44331667])

In [19]:
model.fit(Xr_tp, yr_d, epochs=10,
         validation_data=(Xe_tp, ye_d))

Train on 60000 samples, validate on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 13s 220us/sample - loss: 0.6344 - accuracy: 0.6447 - val_loss: 0.6259 - val_accuracy: 0.6592
Epoch 2/10
60000/60000 [==============================] - 15s 242us/sample - loss: 0.6275 - accuracy: 0.6534 - val_loss: 0.6204 - val_accuracy: 0.6666
Epoch 3/10
60000/60000 [==============================] - 13s 219us/sample - loss: 0.6170 - accuracy: 0.6677 - val_loss: 0.6098 - val_accuracy: 0.6791
Epoch 4/10
60000/60000 [==============================] - 14s 231us/sample - loss: 0.6099 - accuracy: 0.6761 - val_loss: 0.5981 - val_accuracy: 0.6935
Epoch 5/10
60000/60000 [==============================] - 14s 240us/sample - loss: 0.6001 - accuracy: 0.6892 - val_loss: 0.6066 - val_accuracy: 0.6822
Epoch 6/10
60000/60000 [==============================] - 14s 227us/sample - loss: 0.5870 - accuracy: 0.7067 - val_loss: 0.5770 - val_accuracy: 0.7182
Epoch 7/10
60000/60000 [====================

In [20]:
model.fit(Xr_tp, yr_d, epochs=10,
         validation_data=(Xe_tp, ye_d))

Train on 60000 samples, validate on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 14s 235us/sample - loss: 0.5053 - accuracy: 0.7983 - val_loss: 0.5113 - val_accuracy: 0.7887
Epoch 2/10
60000/60000 [==============================] - 13s 214us/sample - loss: 0.4981 - accuracy: 0.8036 - val_loss: 0.4825 - val_accuracy: 0.8217
Epoch 3/10
60000/60000 [==============================] - 13s 217us/sample - loss: 0.4916 - accuracy: 0.8118 - val_loss: 0.4775 - val_accuracy: 0.8285
Epoch 4/10
60000/60000 [==============================] - 13s 210us/sample - loss: 0.4834 - accuracy: 0.8206 - val_loss: 0.4702 - val_accuracy: 0.8370
Epoch 5/10
60000/60000 [==============================] - 15s 243us/sample - loss: 0.4712 - accuracy: 0.8352 - val_loss: 0.4602 - val_accuracy: 0.8461
Epoch 6/10
60000/60000 [==============================] - 14s 226us/sample - loss: 0.4676 - accuracy: 0.8386 - val_loss: 0.4644 - val_accuracy: 0.8397
Epoch 7/10
60000/60000 [====================

In [21]:
model.fit(Xr_tp, yr_d, epochs=10,
         validation_data=(Xe_tp, ye_d))

Train on 60000 samples, validate on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 14s 236us/sample - loss: 0.4409 - accuracy: 0.8674 - val_loss: 0.4413 - val_accuracy: 0.8666
Epoch 2/10
60000/60000 [==============================] - 13s 218us/sample - loss: 0.4392 - accuracy: 0.8680 - val_loss: 0.4249 - val_accuracy: 0.8844
Epoch 3/10
60000/60000 [==============================] - 14s 228us/sample - loss: 0.4344 - accuracy: 0.8737 - val_loss: 0.4285 - val_accuracy: 0.8801
Epoch 4/10
60000/60000 [==============================] - 13s 214us/sample - loss: 0.4320 - accuracy: 0.8762 - val_loss: 0.4851 - val_accuracy: 0.8162
Epoch 5/10
60000/60000 [==============================] - 13s 221us/sample - loss: 0.4282 - accuracy: 0.8798 - val_loss: 0.4299 - val_accuracy: 0.8783
Epoch 6/10
60000/60000 [==============================] - 14s 226us/sample - loss: 0.4246 - accuracy: 0.8834 - val_loss: 0.4154 - val_accuracy: 0.8938
Epoch 7/10
60000/60000 [====================

In [22]:
model.fit(Xr_tp, yr_d, epochs=10,
         validation_data=(Xe_tp, ye_d))

Train on 60000 samples, validate on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 15s 251us/sample - loss: 0.4150 - accuracy: 0.8941 - val_loss: 0.4071 - val_accuracy: 0.9020
Epoch 2/10
60000/60000 [==============================] - 15s 245us/sample - loss: 0.4144 - accuracy: 0.8941 - val_loss: 0.4070 - val_accuracy: 0.9027
Epoch 3/10
60000/60000 [==============================] - 14s 229us/sample - loss: 0.4111 - accuracy: 0.8984 - val_loss: 0.4009 - val_accuracy: 0.9100
Epoch 4/10
60000/60000 [==============================] - 14s 230us/sample - loss: 0.4108 - accuracy: 0.8987 - val_loss: 0.4091 - val_accuracy: 0.8994
Epoch 5/10
60000/60000 [==============================] - 13s 221us/sample - loss: 0.4099 - accuracy: 0.8988 - val_loss: 0.4678 - val_accuracy: 0.8359
Epoch 6/10
60000/60000 [==============================] - 13s 221us/sample - loss: 0.4063 - accuracy: 0.9028 - val_loss: 0.4078 - val_accuracy: 0.9007
Epoch 7/10
60000/60000 [====================

In [29]:
model.save_weights('./data/Models/1min91acc_010920')

In [28]:
model.save('./data/Models/1min91acc_010920') 

INFO:tensorflow:Assets written to: ./data/Models/1min91acc_010920/assets


In [26]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(Xr_tp, yr_d, epochs=10,
         validation_data=(Xe_tp, ye_d))

Train on 60000 samples, validate on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 15s 246us/sample - loss: 0.3786 - accuracy: 0.9351 - val_loss: 0.3805 - val_accuracy: 0.9329
Epoch 2/10
60000/60000 [==============================] - 13s 215us/sample - loss: 0.3762 - accuracy: 0.9374 - val_loss: 0.3803 - val_accuracy: 0.9327
Epoch 3/10
60000/60000 [==============================] - 13s 222us/sample - loss: 0.3753 - accuracy: 0.9383 - val_loss: 0.3810 - val_accuracy: 0.9324
Epoch 4/10
60000/60000 [==============================] - 15s 246us/sample - loss: 0.3747 - accuracy: 0.9387 - val_loss: 0.3798 - val_accuracy: 0.9319
Epoch 5/10
60000/60000 [==============================] - 14s 232us/sample - loss: 0.3742 - accuracy: 0.9392 - val_loss: 0.3793 - val_accuracy: 0.9332
Epoch 6/10
60000/60000 [==============================] - 15s 249us/sample - loss: 0.3735 - accuracy: 0.9403 - val_loss: 0.3790 - val_accuracy: 0.9332
Epoch 7/10
60000/60000 [====================

In [27]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(Xr_tp, yr_d, epochs=10,
         validation_data=(Xe_tp, ye_d))

Train on 60000 samples, validate on 60000 samples
Epoch 1/10
60000/60000 [==============================] - 15s 253us/sample - loss: 0.3689 - accuracy: 0.9456 - val_loss: 0.3763 - val_accuracy: 0.9364
Epoch 2/10
60000/60000 [==============================] - 14s 227us/sample - loss: 0.3684 - accuracy: 0.9465 - val_loss: 0.3764 - val_accuracy: 0.9363
Epoch 3/10
60000/60000 [==============================] - 14s 229us/sample - loss: 0.3682 - accuracy: 0.9467 - val_loss: 0.3769 - val_accuracy: 0.9354
Epoch 4/10
60000/60000 [==============================] - 14s 227us/sample - loss: 0.3682 - accuracy: 0.9466 - val_loss: 0.3761 - val_accuracy: 0.9366
Epoch 5/10
60000/60000 [==============================] - 14s 226us/sample - loss: 0.3681 - accuracy: 0.9474 - val_loss: 0.3760 - val_accuracy: 0.9366
Epoch 6/10
60000/60000 [==============================] - 14s 241us/sample - loss: 0.3680 - accuracy: 0.9471 - val_loss: 0.3759 - val_accuracy: 0.9366
Epoch 7/10
60000/60000 [====================